In [20]:
from module.function import *

In [21]:
from module.learning import *

In [22]:
import pandas as pd

In [45]:
a = pd.read_csv('data_csv/data_ADA_KRW.csv').drop(['Unnamed: 0'], axis = 1)

In [46]:
a

,close,high,low,open,time,volumefrom,volumeto,timestamp,KRW
0,0.000391,0.000393,0.000387,0.000393,1519146000,840465,327.93,2018-02-21 02:00:00,387.840603
1,0.000398,0.000399,0.000389,0.000391,1519149600,1302964,513.33,2018-02-21 03:00:00,393.334405
2,0.000399,0.000403,0.000398,0.000398,1519153200,2461204,984.86,2018-02-21 04:00:00,391.883556
3,0.000397,0.000403,0.000395,0.000399,1519156800,1369060,548.18,2018-02-21 05:00:00,387.893991
4,0.000400,0.000402,0.000396,0.000397,1519160400,845380,336.47,2018-02-21 06:00:00,390.055441
5,0.000393,0.000402,0.000393,0.000400,1519164000,611872,242.66,2018-02-21 07:00:00,372.595762
6,0.000393,0.000396,0.000391,0.000393,1519167600,874594,343.76,2018-02-21 08:00:00,369.571050
7,0.000382,0.000393,0.000382,0.000393,1519171200,1967310,762.40,2018-02-21 09:00:00,361.115450
8,0.000383,0.000386,0.000380,0.000382,1519174800,1489288,569.29,2018-02-21 10:00:00,363.774649
9,0.000383,0.000385,0.000379,0.000383,1519178400,1150122,438.66,2018-02-21 11:00:00,364.934255
